# 预训练数据集格式 

预训练用大规模的无监督语料对模型训练

In [11]:
import json
pretrain_dataset_path = r'D:\MiniMind\dataset\pretrain_hq.jsonl'
with open(pretrain_dataset_path, 'r', encoding='utf-8') as f:
    for line_num, line in enumerate(f, 1):
        data = json.loads(line.strip())
        break

print(data.keys())
print(data)

dict_keys(['text'])
{'text': '<|im_start|>鉴别一组中文文章的风格和特点，例如官方、口语、文言等。需要提供样例文章才能准确鉴别不同的风格和特点。<|im_end|> <|im_start|>好的，现在帮我查一下今天的天气怎么样?今天的天气依据地区而异。请问你需要我帮你查询哪个地区的天气呢？<|im_end|> <|im_start|>打开闹钟功能，定一个明天早上七点的闹钟。好的，我已经帮您打开闹钟功能，闹钟将在明天早上七点准时响起。<|im_end|> <|im_start|>为以下场景写一句话描述：一个孤独的老人坐在公园长椅上看着远处。一位孤独的老人坐在公园长椅上凝视远方。<|im_end|> <|im_start|>非常感谢你的回答。请告诉我，这些数据是关于什么主题的？这些数据是关于不同年龄段的男女人口比例分布的。<|im_end|> <|im_start|>帮我想一个有趣的标题。这个挺有趣的："如何成为一名成功的魔术师" 调皮的标题往往会吸引读者的注意力。<|im_end|> <|im_start|>回答一个问题，地球的半径是多少？地球的平均半径约为6371公里，这是地球自赤道到两极的距离的平均值。<|im_end|> <|im_start|>识别文本中的语气，并将其分类为喜悦、悲伤、惊异等。\n文本：“今天是我的生日！”这个文本的语气是喜悦。<|im_end|>'}


# 准备预训练数据加载器

In [14]:
import json
import torch
from torch.utils.data import Dataset

class PretrainDataset(Dataset):
    def __init__(self, data_path, tokenizer, max_length=512):
        super().__init__()
        self.tokenizer = tokenizer # 分词器，把文本转为token ID
        self.max_length = max_length # 每条样本的最大token长度
        self.samples = self.load_data(data_path) # 加载数据
    def load_data(self, path):
        # 从文件中加载数据，每一行为一条JSON格式的样本
        samples = []
        with open(path, 'r', encoding="utf-8") as f:
            for line_num, line in enumerate(f, 1):
                # 读取每一行，解析成字典结构
                data = json.loads(line.strip())
                samples.append(data)
        return samples
    def __len__(self):
        # 返回样本数量
        return len(self.samples)
    def __getitem__(self, index):
        sample = self.samples[index]
        # 将样本中的文本字段进行tokenize
        encoding = self.tokenizer(
            str(sample['text']),
            max_length=self.max_length,
            padding='max_length',
            truncation=True,
            return_tensors='pt'
        )
        # 获取input_ids张量，去除batch维度
        input_ids = encoding.input_ids.squeeze()

        # 计算loss_mask: pad位置不参与loss
        loss_mask = (input_ids != self.tokenizer.pad_token_id)

        # 用前一个token预测下一个
        X = torch.tensor(input_ids[:-1], dtype=torch.long) # 输入：[0, ..., n-2]
        Y = torch.tensor(input_ids[1:], dtype=torch.long) # 目标：[1, ..., n-1]
        loss_mask = torch.tensor(loss_mask[1:], dtype=torch.long)

        return X, Y, loss_mask

In [15]:
from torch.utils.data import DataLoader
from transformers import AutoTokenizer

max_length = 512
data_path = r"D:\MiniMind\dataset\pretrain_hq.jsonl"
tokenizer = AutoTokenizer.from_pretrained(r"D:\MiniMind\model")
train_ds = PretrainDataset(data_path, tokenizer, max_length)

train_loader = DataLoader(
    train_ds,
    batch_size=2,
    pin_memory=True,
    drop_last=False,
    shuffle=False,
    num_workers=0,
)

In [16]:
print(len(train_loader))

706552


In [17]:
for item in train_loader:
    print([i.shape for i in item])
    break

[torch.Size([2, 511]), torch.Size([2, 511]), torch.Size([2, 511])]


C:\Users\user\AppData\Local\Temp\ipykernel_22028\2296140921.py:40: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.detach().clone() or sourceTensor.detach().clone().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X = torch.tensor(input_ids[:-1], dtype=torch.long) # 输入：[0, ..., n-2]
C:\Users\user\AppData\Local\Temp\ipykernel_22028\2296140921.py:41: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.detach().clone() or sourceTensor.detach().clone().requires_grad_(True), rather than torch.tensor(sourceTensor).
  Y = torch.tensor(input_ids[1:], dtype=torch.long) # 目标：[1, ..., n-1]
C:\Users\user\AppData\Local\Temp\ipykernel_22028\2296140921.py:42: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.detach().clone() or sourceTensor.detach().clone().requires_grad_(True), rather than torch.tensor(sourceTensor).
  loss_mask = torch.tensor(loss_mask[1:], dtype=torch.long)


# 开始预训练

In [ ]:
# 定义交叉熵损失函数
import torch.nn as nn
from contextlib import nullcontext

loss_fct = nn.CrossEntropyLoss(reduction='none')

# 在不改变代码结构的前提下，CPU上什么都不做，GPU上就自动混合精度
# CPU不支持float16加速运算，GPU上可以
ctx = nullcontext() if device_type == 'cpu' else torch.cuda.amp.autocast()

# 遍历训练数据加载器
for step, (X, Y, loss_mask) in enumerate(train_loader):
    X = X.to(args.device)
    Y = Y.to(args.device)
    loss_mask = loss_mask.to(args.device) # 形状都是(batch_size, seq_len)

    # 自定义学习率调度函数更新学习率
    lr = get_lr(
        epoch * iter_per_epoch + step, # 当前训练步数
        args.epochs * iter_per_epoch, # 总训练步数
        args.learning_rate # 初始学习率
    )
    for param_group in optimizer.param_groups:
        param_group['lr'] = lr
    
    # 自动混合精度上下文
    with ctx: 
        res = model(X)

        # 计算token级别的交叉熵损失
        loss = loss_fct(
            res.logits.view(-1, res.logits.size(-1)), # logits是softmax之前未归一化的分数
            Y.view(-1)
        ).view(Y.size())

        # 仅在非pad的位置计算损失
        loss = (loss * loss_mask).sum() / loss_mask.sum()

        # 加入模型可能返回的辅助损失
        loss += res.aux_loss

        # 梯度累计
        loss = loss / args.accumulation_steps

    # PyTorch自带一个自动混合精度模块，在float16混合精度下训练更稳定
    # GPU上直接用autocast 混合精度fp16有时会下溢，有时也上溢inf
    from torch.amp import GradScaler
    scaler = GradScaler()

    # 给loss乘上一个缩放因子，放大再反向传播
    scaler.scale(loss).backward() 

    # 累计到一定步数后进行一次参数更新
    if (step + 1) % args.accumulation_steps == 0:
        # 把刚刚放大的梯度恢复到原本的尺度
        scaler.unscale_(optimizer)
        # 梯度剪裁必须基于真实梯度, 末尾_表示原地操作
        torch.nn.utils.clip_grad_norm_(model.parameters(), args.grad_clip) # （要剪的参数集合， 阈值）

        # 参数更新
        scaler.step(optimizer)

        # 更新scaler的缩放因子S
        scaler.update()

        # 清空梯度，把.grad设置为None(True). False的话是把梯度张量设置为0
        optimizer.zero_grad(set_to_none=True)




: 